In [120]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_validator import ExcelValidator
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from ast_transformation.formula_generator import FormulaGenerator

In [121]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_7'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")

In [122]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)

In [123]:
is_valid = ExcelValidator.validate_excel(excel_reduced)

In [124]:
if not is_valid:
    raise Exception("Excel file is not valid")

In [125]:
excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)

In [126]:
extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)

In [127]:
series_data = SeriesExtractor.extract_table_details(extracted_tables, data)

In [128]:
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)

In [129]:
if not is_compatible:
    raise Exception("Excel file is not compatible")

In [130]:
series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)

In [131]:
series_dict

{'Sheet1': [Series(series_id='Sheet1|App|1|1', worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='App', formulas=[None, None], values=['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
  Series(series_id='Sheet1|Category|1|2', worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='Category', formulas=[None, None], values=['ART_AND_DESIGN', 'ART_AND_DESIGN'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=2, coordinate='B2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
  Series(series_id='Sheet1|Rating|1|3', worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), seri

In [132]:
series_mapping = SeriesMapper.map_series(series_dict)

In [133]:
series_mapping

{Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None): {Cell(column=1, row=2, coordinate=None, value=None, value_type=None): (0,
   Series(series_id='Sheet1|App|1|1', worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='App', formulas=[None, None], values=['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>)),
  Cell(column=1, row=3, coordinate=None, value=None, value_type=None): (1,
   Series(series_id='Sheet1|App|1|1', worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='App', formulas=[None, None], values=['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, ro

In [134]:
series_iterator = SeriesIterator.iterate_series(series_dict)

In [135]:
series_list = [series for series in series_iterator if series.formulas != [None, None]]

Series Id can be concatenation of sheet_name, header_name, header_column_index, header_row_index

In [136]:
for series in series_list:
    formula_1 = series.formulas[0]
    formula_1_ast = FormulaParser.parse_formula(formula_1)
    series_implementer = SeriesImplementer(series_mapping, sheet_name = series.worksheet.sheet_name)
    formula_1_ast_series = series_implementer.replace_range_nodes(formula_1_ast)

    formula_2 = series.formulas[1]
    formula_2_ast = FormulaParser.parse_formula(formula_2)
    formula_2_ast_series = series_implementer.replace_range_nodes(formula_2_ast)


    print(formula_1)
    print(formula_2)

    ast_generator = FormulaGenerator.get_ast_generator(formula_1_ast_series, formula_2_ast_series)
    
    ast_delta = ast_generator.get_ast_with_deltas()
    ast_delta_serialised = SeriesImplementer.serialise_ast_to_formula(ast_delta)
    print(ast_delta_serialised)
    

=SUM(A2,B2)
=SUM(A3,B3)
node and length
(('Sheet1|App|1|1',), (0, 0))
2
node and length
(('Sheet1|Category|1|2',), (0, 0))
2
SUM(('Sheet1|App|1|1',), (1, 1, 0, 0), 0, ('Sheet1|Category|1|2',), (1, 1, 0, 0), 0)
=IF(E2="good","Yes","No")
=IF(E3="good","Yes","No")
node and length
(('Sheet1|Size|1|5',), (0, 0))
2
IF((('Sheet1|Size|1|5',), (1, 1, 0, 0), 0 = "good"), "Yes", "No")
=COUNTIF(B:B,"<0")
=COUNTIF(B:B,"<0")
COUNTIF((('Sheet1|Category|1|2',), (None, None)), "<0")
=SUMIF(D:D,">1000")
=SUMIF(D:D,">1000")
SUMIF((('Sheet1|Reviews|1|4',), (None, None)), ">1000")
=AVERAGEIF(C:C,">0")
=AVERAGEIF(C:C,">0")
AVERAGEIF((('Sheet1|Rating|1|3',), (None, None)), ">0")
=IF(OR(A2>0,B2>0),"Acceptable","Check")
=IF(OR(A3>0,B3>0),"Acceptable","Check")
node and length
(('Sheet1|App|1|1',), (0, 0))
2
node and length
(('Sheet1|Category|1|2',), (0, 0))
2
IF(OR((('Sheet1|App|1|1',), (1, 1, 0, 0), 0 > 0), (('Sheet1|Category|1|2',), (1, 1, 0, 0), 0 > 0)), "Acceptable", "Check")


AttributeError: 'NoneType' object has no attribute 'startswith'